In [1]:
import SignalAnalysis as SA
import DynamicalSystems as DS
import DynamicalSystems.RecurrenceAnalysis as RA
import Random
import PyPlot as plt

#import Clustering

# TODO

1.  DONE Make a histogram for every number of recurrences. DONE
    *   Can be interesting.
2.  Be carefull with sampling. Always use the same percentage when altering series size.
3.  analyse what happens to gamma when resizing the time series.
    *   make a graph
    *   if it tends to a constant value, clustering is probably wrong.

# Generating data and organizing it
#### (with some plots along the way) ####

In [9]:
SERIES_SIZE = 1000
SAMPLE_SIZE = 100000

TRAJECTORY_TYPE = 'PinkGaussian' # Or constant or RedGaussian

# generating stochastic data
# constant distribution
#trajectory = zeros(SERIES_SIZE)
#Random.rand!(trajectory);

# PinkGaussian
trajectory = rand(SA.PinkGaussian(1000))


1000-element Vector{Float64}:
 -0.27566453961776377
 -0.1595384872793164
  0.47426954543845673
  0.2256764819958952
  0.7984840444588129
 -0.1730937282928079
  1.411025896979595
  1.2586883029994278
  0.5371241103682419
  0.6077239636493147
  ⋮
 -1.369434120008417
 -1.3265938919968678
 -1.2312549261931245
 -1.7229132313588327
 -2.917208959724256
 -0.9373667389100737
 -0.9053474978675853
 -1.5278549042417076
 -0.4608791906331504

In [3]:
# getting recurrence matrix
matrix = RA.RecurrenceMatrix(trajectory, 0.28)

new_matrix = zeros((SERIES_SIZE, SERIES_SIZE))
for i in 1:SERIES_SIZE, j in 1:SERIES_SIZE
    new_matrix[i, j] = 1 * matrix[i, j]    
end
matrix = trunc.(new_matrix);

In [4]:
# Getting samples

# defining random indices
x_index = zeros(SAMPLE_SIZE)
y_index = zeros(SAMPLE_SIZE)

Random.rand!(x_index, collect(1:SERIES_SIZE-2))
Random.rand!(y_index, collect(1:SERIES_SIZE-2))

x_index = trunc.(Int, x_index)
y_index = trunc.(Int, y_index);


sample = []
for (x, y) in zip(x_index, y_index)
    push!(sample, matrix[x:x+2, y:y+2])
end;

In [5]:
# This generates all possible matrices with N=3
# It also creates a second list with matrices sorted by class, where the index
# is <number of recurrences> + 1

# making 9 fors. I'm sorry.
possible_matrices = []
for a in 0:1, b in 0:1, c in 0:1,
    d in 0:1, e in 0:1, f in 0:1,
    g in 0:1, h in 0:1, i in 0:1

    push!(possible_matrices, [a b c; d e f; g h i])
end

possible_matrices_by_class = [[] for i in 1:10]

for i in possible_matrices, j in 1:10
    if sum(i) == j-1
        push!(possible_matrices_by_class[j], i)
    end
end

In [6]:
# comparing sample with possible_matrices
# this also sorts the obtained matrices into their respective classes
counts = zeros(size(possible_matrices))
sorted_by_class = [[] for i in 1:10]

sample_copy = copy(sample)

while size(sample_copy)[1] > 0
    current_matrix = pop!(sample_copy)
    push!(sorted_by_class[trunc(Int, sum(current_matrix) + 1)], current_matrix)
    for (i, j) in enumerate(possible_matrices)
        if current_matrix == j
            counts[i] += 1
            break
        end
    end

end


In [7]:
# counting, for each class, the ocurrences of each microstate
counts_by_class = [[] for i in 1:10]

for (class, possible_iter) in enumerate(possible_matrices_by_class)
    counts_iter = zeros(size(possible_iter))
    for k in sorted_by_class[class]
        for (i, j) in enumerate(possible_iter)
            if k == j
                counts_iter[i] += 1
                break
            end
        end
    end
    push!(counts_by_class[class], counts_iter)
end

In [8]:
# Saving some data
open("test.txt", "w") do file
    for i in counts_by_class
        write(file, "$(collect(i...))\n")
    end
end

open("../matrices.txt", "w") do file
    for i in possible_matrices_by_class
        write(file, "$(i)\n")
    end
end
    

In [9]:
for i in 1:size(possible_matrices_by_class)[1]
    println("$(i),  $(size(possible_matrices_by_class[i]))")
end

1,  (1,)
2,  (9,)
3,  (36,)
4,  (84,)
5,  (126,)
6,  (126,)
7,  (84,)
8,  (36,)
9,  (9,)
10,  (1,)


In [10]:
for i in 1:10
    fig = plt.figure()
    plt.suptitle("$(i-1) recurrences")
    plt.hist(counts_by_class[i], 40)
    
end

In [ ]:
plt.close()
plt.hist(counts, 300);

In [12]:
plt.close()
plt.scatter(collect(1:512), counts, s=.5)

PyObject <matplotlib.collections.PathCollection object at 0x7f3802176ca0>

In [13]:
for i in 1:10
    fig = plt.figure()
    plt.suptitle("$(i-1) recurrences")
    plt.scatter(collect(1:size(collect(counts_by_class[i]...))[1]), counts_by_class[i], s=.75);
end

# Clusterizing it (Probably)

In [14]:
# the function Clustering.hclust needs the distances pre-calculated

# testing for 4 recurrences
d_size = size(collect(counts_by_class[4]...))[1]
distance_matrix = zeros((d_size, d_size))

for (i, iitem) in enumerate(counts_by_class[4]...),
    (j, jitem) in enumerate(counts_by_class[4]...)
    
    distance_matrix[i, j] = abs(iitem - jitem)
end